- (Y) remove all rows where customer id is missing - **SQ**L or Python
- (Y) remove certain stock codes: bank charges, post, d, m cruk - **SQL** or Python
- (Y) add description to the online transactions table - **SQL** or Python
- (Y) remove all duplicate data - Python
- (Y) replace missing stock description with Unknown - SQL or **Python**
- (Y) fix data type for the invoice date column - **Python** or SQL

In [99]:
# import all the libraries we need

import psycopg2
import pandas as pd

# add if you want to remove warning messages
import warnings
warnings.filterwarnings("ignore")

In [100]:
# you do not need to import these libraries - you just hardcode the variables

import os

from dotenv import load_dotenv
load_dotenv()  # only for local testing


dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [101]:
# connect to redshift
# reference for the connect function: https://www.psycopg.org/docs/module.html

def connect_to_redshift(dbname, host, port, user, password):

    connect = psycopg2.connect(
                dbname=dbname, 
                host=host,
                port=port, 
                user=user, 
                password=password
            )
    
    print("Connection was successful")

    return connect
    

In [102]:
connect = connect_to_redshift(dbname, host, port, user, password)

Connection was successful


In [103]:
# check everything is working

query = """
select *
from bootcamp.stock_description
limit 10
"""

pd.read_sql(query, connect)

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
5,10124G,ARMY CAMO BOOKCOVER TAPE
6,10125,MINI FUNKY DESIGN TAPES
7,10133,COLOURING PENCILS BROWN TUBE
8,10135,COLOURING PENCILS BROWN TUBE
9,11001,ASSTD DESIGN RACING CAR PEN


In [37]:
# double check how many rows we originally have

query = """
select count(*)
from bootcamp.online_transactions ot
"""

pd.read_sql(query, connect)

,count
0,541910


In [38]:
# task1: remove all rows where customer is missing
# task 2: remove all stock codes that are called bank charges, post, d, m cruk

query = """
select ot.*
from bootcamp.online_transactions ot
where customer_id <> ''
 and ot.stock_code not in ('BANK CHARGES', 'POST', 'D', 'M', 'CRUK')
"""

online_trans = pd.read_sql(query, connect)

In [39]:
online_trans.shape

(405063, 7)

In [53]:
# task3: join description to the online transactions table

query = """
select ot.*,
       sd.description
from bootcamp.online_transactions ot
left join bootcamp.stock_description sd on ot.stock_code = sd.stock_code
where customer_id <> ''
 and ot.stock_code not in ('BANK CHARGES', 'POST', 'D', 'M', 'CRUK')
"""

online_trans = pd.read_sql(query, connect)

In [54]:
online_trans.shape

# we have more rows now because of stock codes having multiple descriptions

(412922, 8)

In [55]:
online_trans[(online_trans.invoice == "536477") &
(online_trans.stock_code == "22423")]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
103,536477,22423,16,2010-12-01 12:27:00,10.95,u1621,United Kingdom,REGENCY CAKESTAND 3 TIER
104,536477,22423,16,2010-12-01 12:27:00,10.95,u1621,United Kingdom,?


In [56]:
# list of stock codes that have more than one description
query = """
select stock_code,
       count(*)
from bootcamp.stock_description
group by stock_code
having count(*) > 1
"""

check = pd.read_sql(query, connect)
check.head()

,stock_code,count
0,16020C,2
1,16207B,2
2,21145,2
3,21232,2
4,21368,2


In [72]:
query = """select *
from bootcamp.stock_description
where stock_code in (select stock_code
from bootcamp.stock_description
group by stock_code
having count(*) > 1)
order by stock_code desc
"""

pd.read_sql(query, connect)

,stock_code,description
0,90210A,GREY ACRYLIC FACETED BANGLE
1,90210A,?
2,90175A,WHITE GLASS CHUNKY CHARM BRACELET
3,90175A,?
4,90125D,PURPLE BERTIE GLASS BEAD BAG CHARM
...,...,...
89,21145,ANTIQUE GLASS PLACE SETTING
90,16207B,PINK HEART RED HANDBAG
91,16207B,?
92,16020C,CLEAR STATIONERY BOX SET


In [57]:
# number of stock codes with multiple description
# you can use python or sql to get this information
check.shape[0]

47

In [76]:
# task3: join description to the online transactions table and remove ? from the stock description table

query = """
select ot.*,
       sd.description
from bootcamp.online_transactions ot
left join (select *
          from bootcamp.stock_description
          where description <> '?') sd on ot.stock_code = sd.stock_code
where ot.customer_id <> ''
 and ot.stock_code not in ('BANK CHARGES', 'POST', 'D', 'M', 'CRUK')
"""

online_trans = pd.read_sql(query, connect)

In [77]:
# now we have the same number of expected rows

online_trans.shape

(405063, 8)

In [121]:
# you can also write this sql query by joining and then adding a where condition to remove the ?

query = """
select ot.*,
       sd.description
from bootcamp.online_transactions ot
left join bootcamp.stock_description sd on ot.stock_code = sd.stock_code
where ot.customer_id <> ''
 and ot.stock_code not in ('BANK CHARGES', 'POST', 'D', 'M', 'CRUK')
 and (sd.description is null or sd.description <> '?')
"""

online_trans = pd.read_sql(query, connect)

In [122]:
online_trans.shape

(405063, 8)

In [83]:
# if you want to drop all occurences of duplicated rows, use keep=False
# this means you drop the row that has duplicated data

online_trans.drop_duplicates(keep=False)

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,84029E,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.
1,536370,22900,24,2010-12-01 08:45:00,2.95,u12583,France,SET 2 TEA TOWELS I LOVE LONDON
2,536373,21730,6,2010-12-01 09:02:00,4.25,u1785,United Kingdom,GLASS STAR FROSTED T-LIGHT HOLDER
3,536375,84406B,8,2010-12-01 09:32:00,2.75,u1785,United Kingdom,CREAM CUPID HEARTS COAT HANGER
4,536378,21212,120,2010-12-01 09:37:00,0.42,u14688,United Kingdom,PACK OF 72 RETROSPOT CAKE CASES
...,...,...,...,...,...,...,...,...
405058,581580,22698,1,2011-12-09 12:20:00,2.95,u12748,United Kingdom,PINK REGENCY TEACUP AND SAUCER
405059,581584,20832,72,2011-12-09 12:25:00,0.72,u13777,United Kingdom,RED FLOCK LOVE HEART PHOTO FRAME
405060,581585,22178,12,2011-12-09 12:31:00,1.95,u15804,United Kingdom,VICTORIAN GLASS HANGING T-LIGHT
405061,581585,84692,25,2011-12-09 12:31:00,0.42,u15804,United Kingdom,BOX OF 24 COCKTAIL PARASOLS


In [78]:
# remove the duplicated rows data

# check how many duplicated rows
online_trans.duplicated().sum()

5222

In [81]:
# drop the duplicated rows - but want to keep the first appearance

online_trans_cleaned = online_trans.drop_duplicates(keep='first')

In [82]:
online_trans_cleaned.shape

(399841, 8)

In [84]:
online_trans_cleaned.duplicated().sum()

0

In [85]:
# identify missing data

online_trans_cleaned.isnull().sum()

invoice            0
stock_code         0
quantity           0
invoice_date       0
price              0
customer_id        0
country            0
description     1172
dtype: int64

In [86]:
# replace the missing values with Unknown

online_trans_cleaned.description.fillna("Unknown", inplace=True)

In [87]:
# check if you have any missing data

online_trans_cleaned.isnull().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
description     0
dtype: int64

In [88]:
# fix the data type
online_trans_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399841 entries, 0 to 405062
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       399841 non-null  object 
 1   stock_code    399841 non-null  object 
 2   quantity      399841 non-null  int64  
 3   invoice_date  399841 non-null  object 
 4   price         399841 non-null  float64
 5   customer_id   399841 non-null  object 
 6   country       399841 non-null  object 
 7   description   399841 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 27.5+ MB


In [91]:
# use the to datetime function from pandas to fix the datetime

online_trans_cleaned.invoice_date = pd.to_datetime(online_trans_cleaned.invoice_date)

In [92]:
# check the info

online_trans_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 399841 entries, 0 to 405062
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       399841 non-null  object        
 1   stock_code    399841 non-null  object        
 2   quantity      399841 non-null  int64         
 3   invoice_date  399841 non-null  datetime64[ns]
 4   price         399841 non-null  float64       
 5   customer_id   399841 non-null  object        
 6   country       399841 non-null  object        
 7   description   399841 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.5+ MB


In [94]:
online_trans_cleaned.shape

(399841, 8)

In [93]:
# Activity: Load the transformed data frame to s3